In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
import pandas as pd
import numpy as np

#Load dataset
path = '/content/gdrive/MyDrive/Emotion Detection/Data/fer2013.csv'
df = pd.read_csv(path)

#Split data into training and testing set
train_x, train_y, test_x, test_y = [], [], [], []
for index, row in df.iterrows():
    val = row['pixels'].split(' ')
    try:
        if 'Training' in row['Usage']:
            train_x.append(np.array(val, 'float32'))
            train_y.append(row['emotion'])
        elif 'PublicTest' in row['Usage']:
            test_x.append(np.array(val, 'float32'))
            test_y.append(row['emotion'])
    except:
        print('Error occured at index: {} and row: {}'.format(index, row))

print('Training data (length: {}): '.format(len(train_x)))
print(train_x)
print('Training label (length: {}): '.format(len(train_y)))
print(train_y)

In [ ]:
from tensorflow import keras
from keras.utils import np_utils

train_x = np.array(train_x, 'float32')
train_y = np.array(train_y, 'float32')
test_x = np.array(test_x, 'float32')
test_y = np.array(test_y, 'float32')

train_y = np_utils.to_categorical(train_y, num_classes = 7)
test_y = np_utils.to_categorical(test_y, num_classes = 7)

train_x /= 255.0
train_x -= 0.5
train_x *= 2.0
test_x /= 255.0
test_x -= 0.5
test_x *= 2.0

train_x = train_x.reshape(train_x.shape[0], 48, 48, 1)
test_x = test_x.reshape(test_x.shape[0], 48, 48, 1)

print('Training data (length: {}): '.format(len(train_x)))
print(train_x)
print('Training label (length: {}): '.format(len(train_y)))
print(train_y)

In [4]:
#Split training set into training and validation data
length_samples = len(train_x)
length_train_samples = int((1 - 0.2) * length_samples)

train_data = train_x[:length_train_samples]
train_label = train_y[:length_train_samples]

val_x = train_x[length_train_samples:]
val_y = train_y[length_train_samples:]
val_data = (val_x, val_y)

In [ ]:
from keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout, BatchNormalization
from keras.models import Sequential

model = Sequential()

#Add the first Block
model.add(Conv2D(filters = 32, kernel_size = (3, 3), padding = 'same', activation = 'relu', input_shape = train_data.shape[1:]))
model.add(BatchNormalization())
model.add(Conv2D(filters = 32, kernel_size = (3, 3), padding = 'same', activation = 'relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size = (2, 2)))
model.add(Dropout(0.2))

#Add the second Block
model.add(Conv2D(filters = 64, kernel_size = (3, 3), padding = 'same', activation = 'relu'))
model.add(BatchNormalization())
model.add(Conv2D(filters = 64, kernel_size = (3, 3), padding = 'same', activation = 'relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size = (2, 2)))
model.add(Dropout(0.2))

#Add the third Block
model.add(Conv2D(filters = 128, kernel_size = (3, 3), padding = 'same', activation = 'relu'))
model.add(BatchNormalization())
model.add(Conv2D(filters = 128, kernel_size = (3, 3), padding = 'same', activation = 'relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size = (2, 2)))
model.add(Dropout(0.2))

#Add the four Block
model.add(Conv2D(filters = 256, kernel_size = (3, 3), padding = 'same', activation = 'relu'))
model.add(BatchNormalization())
model.add(Conv2D(filters = 256, kernel_size = (3, 3), padding = 'same', activation = 'relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size = (2, 2)))
model.add(Dropout(0.2))

#Add the Flatten layer
model.add(Flatten())

#Add the ouput layer
model.add(Dense(units = 256, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(units = 64, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(units = 7, activation = 'softmax'))

#Display model summary
model.summary()

In [6]:
from keras.preprocessing.image import ImageDataGenerator

data_generator = ImageDataGenerator(
                        featurewise_center = False, 
                        featurewise_std_normalization = False,
                        rotation_range = 10,
                        width_shift_range = 0.1,
                        height_shift_range = 0.1,
                        zoom_range = 0.1,
                        horizontal_flip = True)

In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

#Compile the model
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

#Early stop training
early_stopping = EarlyStopping(monitor = 'val_loss', patience = 10, verbose = 1)

#Save the best model
best_model = ModelCheckpoint(filepath = 'Emotion_Detection_bestmodel.h5', monitor = 'val_loss', verbose = 1, save_best_only = True)

#Reduce learning rate
reduce_lr = ReduceLROnPlateau(monitor = 'val_loss', factor = 0.2, patience = 5, verbose = 1, min_lr = 0.0001)

#Start training
model_history = model.fit(
                        data_generator.flow(train_data, train_label, batch_size = 64),
                        epochs = 50,
                        validation_data = val_data,
                        callbacks = [early_stopping, best_model, reduce_lr],
                        shuffle = True)

In [ ]:
#Load the best model
keras.models.load_model(filepath = 'Emotion_Detection_bestmodel.h5')

model.evaluate(test_x, test_y)

In [ ]:
import matplotlib.pyplot as plt

# Get training loss and validation loss from model history
history_dict = model_history.history
loss = history_dict['loss']
val_loss = history_dict['val_loss']

# Diplay a chart of training loss and validation loss
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss)
plt.plot(epochs, val_loss)

plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(['Training loss', 'Val loss'], loc='center right')

plt.show()

In [ ]:
# Get training accuracy and validation loss from model history
history_dict = model_history.history
accuracy = history_dict['accuracy']
val_accuracy = history_dict['val_accuracy']

# Diplay a chart of training accuracy and validation accuracy
epochs = range(1, len(accuracy) + 1)
plt.plot(epochs, accuracy)
plt.plot(epochs, val_accuracy)

plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(['Training accuracy', 'Val accuracy'], loc='center right')

plt.show()